In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [2]:
dif_sta = {'TAM':6, 'GNB':21, 'NOR':26, 'KAN':31, 'BAL':18, 'WAS':15, 'NYJ':4, 'CIN':16, 'LAC':13, 'NWE':32, 'LAR':22,
           'MIN':29, 'IND':12, 'SEA':28, 'MIA':7, 'DEN':20, 'CAR':27, 'DAL':23, 'CHI':7, 'NYG':5, 'JAX':3, 'HOU':19,
           'OAK':9, 'PIT':30, 'ARI':17, 'ATL':24, 'BUF':10, 'CLE':1, 'PHI':25, 'DET':14, 'TEN':11, 'SFO':2}

In [3]:
dif_pass_def = {'TAM':7, 'GNB':21, 'NOR':4, 'KAN':2, 'BAL':28, 'WAS':18, 'NYJ':9, 'CIN':1, 'LAC':24, 'NWE':11, 'LAR':19,
           'MIN':30, 'IND':17, 'SEA':16, 'MIA':12, 'DEN':13, 'CAR':15, 'DAL':20, 'CHI':26, 'NYG':10, 'JAX':31, 'HOU':5,
           'OAK':14, 'PIT':23, 'ARI':29, 'ATL':6, 'BUF':1, 'CLE':8, 'PHI':3, 'DET':25, 'TEN':27, 'SFO':22}

In [4]:
def change_rank(x):
    return (-1*(x-32))+1


In [32]:
def clean_stats_qb(run_stats, dic_def):
    run_stats = run_stats.rename({'Unnamed: 7': 'Where'}, axis=1)
    run_stats = run_stats[(run_stats['Pos'] == 'QB') & (run_stats['Att'] >= 10)]
    result = []
    points_home = []
    points_away = []
    for x in run_stats.Result:
        result.append(x[0])
        reg1 = re.findall(r'\d*[^-]', x)
        points_home.append(reg1[2])
        points_away.append(reg1[3])
    run_stats['Result'] = result
    run_stats['Points_Team'] = points_home
    run_stats['Points_Opp'] = points_away
    run_stats.Where.fillna('home', inplace=True)
    run_stats2 = run_stats.reset_index(drop=True)
    stadium = [] 
    for i, x in enumerate(run_stats2.Where):
        if x == 'home':
            stadium.append(1)
        else:
            stadium.append(0)
    run_stats2['Where'] = stadium
    dif_list = []
    for i, x in enumerate(run_stats2.Where):
        if x == 0:
            dif_list.append(dif_sta[run_stats2.Opp[i]]+32)
        else:
            dif_list.append(change_rank(dif_sta[run_stats2.Tm[i]]))
    run_stats2['Stad_Diff'] = dif_list
    run_stats3 = run_stats2.drop(['Lg', 'Tm', 'G#', 'Rk', 'Pos', 'Age', 'Y/A', 'Yds.1', 'Rate', 'Cmp', 'AY/A'], axis=1)
    run_stats3['Month'] = [x.month for x in run_stats3.Date]
    run_stats3['Month'] = run_stats3['Date'].map({9:'September', 10:'October', 11:'November', 12:'December'})
    run_stats3['Points_Team'] = pd.to_numeric(run_stats3.Points_Team, errors='coerce')
    run_stats3['Points_Opp'] = pd.to_numeric(run_stats3.Points_Opp, errors='coerce')
    run_stats3['Result'] = run_stats3.Result.map({'W':1, 'L':0, 'T':2})
    run_stats3['Opp'] = run_stats3['Opp'].map(dic_def)
    run_stats4 = run_stats3.set_index(['Date', 'Week', 'Player'])
    run_stats5 = pd.get_dummies(run_stats4)
    return run_stats5

In [33]:
def clean_stats_qb19_w4(df1, w1):
    w1['Date_October'] = [0 for x in range(len(w1))]
    w1['Date_November'] = [0 for x in range(len(w1))]
    w1['Date_December'] = [0 for x in range(len(w1))]
    w1['Day_Sat'] = [0 for x in range(len(w1))]
    w1 = w1[list(df1.columns)]
    run = pd.concat([df1, w1], sort=False)
    return run

In [34]:
pass1 = pd.read_excel('../data/pass/sportsref_pass1_18.xls')
for x in range(1,12):
    c = pd.read_excel(f'../data/pass/sportsref_downloadp({x}).xls', skiprows=1)
    pass1 = pd.concat([pass1, c], ignore_index = True, sort=False)

In [35]:
passw1 = pd.read_excel('../data/pass/sportsref_downloadpred(1).xls', skiprows=1)

In [36]:
pass_stats1 = clean_stats_qb(pass1, dif_pass_def)

In [37]:
pass_w1 = clean_stats_qb(passw1, dif_pass_def)

In [38]:
pass_stats19 = clean_stats_qb19_w4(pass_stats1, pass_w1)

In [41]:
pass_stats19.TD.max()

6